In [1]:
# taken from ngboost
# https://github.com/stanfordmlgroup/ngboost/blob/master/figures/toy.py
import numpy as np


np.random.seed(1)
def gen_data(n=50, bound=1, deg=3, beta=1, noise=0.9, intcpt=-1):
    x = np.linspace(-bound, bound, n)[:, np.newaxis]
    h = np.linspace(-bound, bound, n)[:, np.newaxis]
    e = np.random.randn(*x.shape) * (0.1 + 10 * np.abs(x))
    y = 50 * (x**deg) + h * beta + noise * e + intcpt
    return x, y.squeeze(), np.c_[h, np.ones_like(h)]

# training
x_tr, y_tr, _ = gen_data(n=100)
# Prepend a bias column to x_tr
# adapted from ngboost for clarity
#from sklearn.preprocessing import PolynomialFeatures
#poly_transform = PolynomialFeatures(1)
#x_tr = poly_transform.fit_transform(x_tr)
x_tr = np.hstack([np.ones((x_tr.shape[0], 1)), x_tr])


x_te, y_te, _ = gen_data(n=1000, bound=1.3)
x_te = np.hstack([np.ones((x_te.shape[0], 1)), x_te])

In [2]:
BLK = 2

# Default arguments
n_estimators = 1 + BLK * 100
lr = 0.03
#lr = 1
minibatch_frac = 0.1
natural = True


# helper
blk = int(n_estimators / 100)

# Model

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

$$f_\theta(y) \sim \mathcal{N}(\mu, \sigma^2)$$
$$\mathcal{L}(\theta, y) := -\log(f_\theta(y))$$

In [4]:
# taken from ngboost
def get_tree_learner():
    tree_learner = DecisionTreeRegressor(
        criterion="friedman_mse",
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_depth=3,
        splitter="best",
        random_state=None,
    )
    #tree_learner = DecisionTreeRegressor(max_depth=2)
    return tree_learner

In [5]:
sigma_threshold = 1e-5
n_iterations = n_estimators
n_samples = len(x_tr)
n_thetas = 2
predictions = np.zeros((n_samples, n_thetas,  n_iterations))

theta_0 = np.mean(y_tr)
theta_1 = np.std(y_tr)

predictions[:, 0, 0] = theta_0
predictions[:, 1, 0] = theta_1

mu = np.ones_like(y_tr) * theta_0
sigma = np.ones_like(y_tr) * theta_1

mu_models = []
sigma_models = []

class NullModel:
    pass

mu_model = NullModel()
sigma_model = NullModel()
mu_model.predict = lambda x: np.ones((len(x),)) * theta_0
sigma_model.predict = lambda x: np.ones((len(x),)) * theta_1
mu_models.append(mu_model)
sigma_models.append(sigma_model)


# Perform gradient boosting for n_iterations
for t in range(1, n_iterations):
    # Calculate gradients
    with np.errstate(divide='ignore'):
        if natural:
            dmu = -(y_tr - mu)
            dsigma = sigma/2 - ((y_tr - mu)**2) / (2 * sigma)
        else:
            dmu = -(y_tr - mu) / (sigma**2)
            dsigma = 1 / sigma - ((y_tr - mu)**2) / (sigma**3)

    dmu[sigma < sigma_threshold] = 0
    dsigma[sigma < sigma_threshold] = 0

    # Fit the weak learner (shallow decision tree) to the negative gradient
    mu_model = get_tree_learner()
    sigma_model = get_tree_learner()

    mu_model.fit(x_tr, dmu)
    sigma_model.fit(x_tr, dsigma)
    mu_models.append(mu_model)
    sigma_models.append(sigma_model)

    # Predictions from the weak learners
    mu_eps = mu_model.predict(x_tr)
    sigma_eps = sigma_model.predict(x_tr)

    mu = mu - lr * mu_eps # == predictions[:, 0, t - 1] + lr * mu_eps
    sigma = sigma - lr * sigma_eps # == predictions[:, 1, t - 1] + lr * sigma_eps
    sigma[sigma <= 0] = 0

    # Update model
    predictions[:, 0, t] = mu
    predictions[:, 1, t] = sigma

In [ ]:
# Now we convert this into two pandas DataFrames for better presentation
df_mu = pd.DataFrame(predictions[:, 0, :], columns=[f"mu_{i}" for i in range(n_iterations)])
df_mu["y"] = y_tr  # Adding the target values as a column

# Display the result
display(df_mu)

df_sigma = pd.DataFrame(predictions[:, 1, :], columns=[f"sigma_{i}" for i in range(n_iterations)])

# Display the result
display(df_sigma)

In [ ]:
df_mu.iloc[:, 3].unique()

In [8]:
def get_all_mu_preds(x):
    preds = [model.predict(x) for model in mu_models]
    df = pd.DataFrame(preds).T
    df.iloc[:, 1:] = (-lr * df.iloc[:, 1:])
    df = df.cumsum(axis=1)
    return df

def get_all_sigma_preds(x):
    preds = [model.predict(x) for model in sigma_models]
    df = pd.DataFrame(preds).T
    df.iloc[:, 1:] = (-lr * df.iloc[:, 1:])
    df = df.cumsum(axis=1)
    return df

# Plot

In [9]:
import matplotlib.pyplot as plt

In [ ]:
mu_preds = get_all_mu_preds(x_te)
sigma_preds = get_all_sigma_preds(x_te)
mu = mu_preds.iloc[:, -1]
sigma = sigma_preds.iloc[:, -1]
plt.figure(figsize=(6, 3))
plt.scatter(x_tr[:, -1], y_tr, color="black", marker=".", alpha=0.5)
plt.plot(x_te[:, -1], mu, color="black", linestyle="-", linewidth=1)
plt.plot(
    x_te[:, -1],
    mu - 1.96 * sigma,
    color="black",
    linestyle="--",
    linewidth=0.3,
)
plt.plot(
    x_te[:, -1],
    mu + 1.96 * sigma,
    color="black",
    linestyle="--",
    linewidth=0.3,
)
plt.ylim([y_te.min()-2, y_te.max()+2])
plt.axis("off")
plt.title(
    "%s Gradient: 100%% fit"
    % ("Natural" if natural else "Ordinary")
)
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt


mu_preds = get_all_mu_preds(x_te)
sigma_preds = get_all_sigma_preds(x_te)


filenames = []
for i in range(n_estimators):
    if i % blk != 0:
        continue
    mu = mu_preds.iloc[:, i]
    sigma = sigma_preds.iloc[:, i]
    plt.figure(figsize=(6, 3))
    plt.scatter(x_tr[:, -1], y_tr, color="black", marker=".", alpha=0.5)
    plt.plot(x_te[:, -1], mu, color="black", linestyle="-", linewidth=1)
    plt.plot(
        x_te[:, -1],
        mu - 1.96 * sigma,
        color="black",
        linestyle="--",
        linewidth=0.3,
    )
    plt.plot(
        x_te[:, -1],
        mu + 1.96 * sigma,
        color="black",
        linestyle="--",
        linewidth=0.3,
    )
    plt.ylim([-75, 75])
    plt.axis("off")
    plt.title(
        "%s Gradient: %02d%% fit"
        % ("Natural" if natural else "Ordinary", i / blk)
    )
    plt.tight_layout()

    filenames.append("./figures/anim/toy%d.png" % i)
    print("Saving ./figures/anim/toy%d" % i)
    plt.savefig("./figures/anim/toy%d.png" % i)
    if i % 100 == 0:
        plt.savefig("./figures/anim/toy%d.pdf" % i)



In [12]:
# create a gif
import imageio

images = []
for filename in filenames:
    images.append(imageio.v2.imread(filename))
for _ in range(10):
    images.append(imageio.v2.imread(filename))
imageio.mimsave(f"./figures/toy_{'natural' if natural else 'ordinary'}_no_ng.gif", images, fps=5)